# HW3: Степенной метод + МОИ

In [474]:
import numpy as np
from matplotlib import pyplot as plt

## Реализация степенного метода

In [475]:
def power_method(A, x0, max_iter, tol):
    x = x0.copy().astype(float) 
    x = x / np.linalg.norm(x)    
    
    lambda_prev = 0              
    iterations = 0
    
    #print(f"{'Итерация':<8} {'Собств. значение':<16} {'Погрешность':<12}")
    #print("-" * 40)
    
    for k in range(max_iter):
        u_k1 = A @ x  # u^(k+1) = A * u^k
        
        numerator = np.dot(u_k1, x)    # (u^(k+1), u^k)
        denominator = np.dot(x, x)     # (u^k, u^k)
        lambda_max = numerator / denominator # Отношение Рэлея
        
        x_new = u_k1 / np.linalg.norm(u_k1)
        
        error = abs(lambda_max - lambda_prev)

        #print(f"{k+1:<8} {lambda_max:<16.8f} {error:<12.2e}")
        
        if error < tol:
            iterations = k + 1
            print("Сошлась за ", iterations, " итераций:")
            print("ERROR = ", error)
            break
        
        x = x_new
        lambda_prev = lambda_max
        iterations = k + 1
    if (iterations == 1000):
        print("Не сошлась за 1000 итераций:")

    return lambda_max, x, iterations

def check(A, lambda_val, eigenvector, tol):
    
    print("\n1. СРАВНЕНИЕ С NUMPY.LINALG.EIG:")
    eigenvalues_np, eigenvectors_np = np.linalg.eig(A)
    max_eig_np = np.max(eigenvalues_np)
    
    print(f"   Разница c NumPy λ = {abs(lambda_val - max_eig_np):.2e}")

    print("2. ПРОВЕРКА: A·x = λ·x")
    left_side = A @ eigenvector
    right_side = lambda_val * eigenvector
    
    residual = left_side - right_side
    error = np.linalg.norm(residual)
    print(f"   Норма невязки = {error:.2e}")
    return error

## Ввод

In [476]:
def load_matrix_from_file(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
    data = []
    for line in lines:
        if line.strip() == '':
            continue
        row = [float(x) for x in line.split()]
        data.append(row)
    A = np.array(data, dtype=float)
    return A

print("Загружаем матрицы...")
matrices = {}
matrix_files = {
    "good1": "hw3/good1.txt", # Матрица с ярко выраженным доминирующим собственным значением
    "good2": "hw3/good2.txt", # Матрица с ярко выраженным доминирующим собственным значением
    "bad1": "hw3/bad1.txt", # Симметричная с двумя наибольшими по модулю собственными значениями
    "bad2": "hw3/bad2.txt", # Жорданова клетка (нелинейный элементарный делитель)
}

for name, filename in matrix_files.items():
    matrices[name] = load_matrix_from_file(filename)
print("Матрицы успешно загружены")

Загружаем матрицы...
Матрицы успешно загружены


## Результаты

In [477]:
max_iter = 1000
precision = 1e-12

print("Точность: ", precision)

for name, matrix in matrices.items():
    print(f"\n{'='*60}")
    print(f"АНАЛИЗ МАТРИЦЫ {name.upper()}")
    print(f"{'='*60}")
    
    dim = matrix.shape
    print(f"Размерность матрицы: {dim}")
    
    initial_vector = np.arange(1, dim[0] + 1, dtype=float)
    lambda_val, eigenvector, iterations = power_method(matrix, initial_vector, max_iter, precision)
    
    check(matrix, lambda_val, eigenvector, precision)
    
    spectral_radius = abs(lambda_val)
    print(f"Спектральный радиус: {spectral_radius:.6f}")


Точность:  1e-12

АНАЛИЗ МАТРИЦЫ GOOD1
Размерность матрицы: (20, 20)
Сошлась за  25  итераций:
ERROR =  4.334310688136611e-13

1. СРАВНЕНИЕ С NUMPY.LINALG.EIG:
   Разница c NumPy λ = 1.42e-13
2. ПРОВЕРКА: A·x = λ·x
   Норма невязки = 1.20e-06
Спектральный радиус: 20.000000

АНАЛИЗ МАТРИЦЫ GOOD2
Размерность матрицы: (20, 20)
Сошлась за  16  итераций:
ERROR =  5.790923296444817e-13

1. СРАВНЕНИЕ С NUMPY.LINALG.EIG:
   Разница c NumPy λ = 7.11e-14
2. ПРОВЕРКА: A·x = λ·x
   Норма невязки = 2.96e-13
Спектральный радиус: 18.000000

АНАЛИЗ МАТРИЦЫ BAD1
Размерность матрицы: (20, 20)
Сошлась за  599  итераций:
ERROR =  9.823253321883385e-13

1. СРАВНЕНИЕ С NUMPY.LINALG.EIG:
   Разница c NumPy λ = 2.86e-11
2. ПРОВЕРКА: A·x = λ·x
   Норма невязки = 1.38e-06
Спектральный радиус: 3.977662

АНАЛИЗ МАТРИЦЫ BAD2
Размерность матрицы: (20, 20)
Не сошлась за 1000 итераций:

1. СРАВНЕНИЕ С NUMPY.LINALG.EIG:
   Разница c NumPy λ = 3.87e-02
2. ПРОВЕРКА: A·x = λ·x
   Норма невязки = 8.91e-05
Спектральный рад

## Реализация метода обратной итерации

In [478]:
def inverse_iteration(A, a, x0, max_iter, tol):
    n = A.shape[0]
    x = x0 / np.linalg.norm(x0) 
    E = np.eye(n) 
    
    lambda_prev = 0
    
    #print(f"{'Итерация':<8} {'λ_approx':<12} {'Погрешность':<12}")
    #print("-" * 40)
    
    for k in range(max_iter):
        # Решаем СЛАУ (A - aE)·y = x
        # эквивалентно y = (A - aE)⁻¹·x
        y = np.linalg.solve(A - a * E, x)
        
        # Отношение Рэлея для исходной матрицы A
        lambda_approx = np.dot(x, A @ x) / np.dot(x, x)
        x_new = y / np.linalg.norm(y)
        
        error = abs(lambda_approx - lambda_prev)
        
        #print(f"{k+1:<8} {lambda_approx:<12.6f} {error:<12.2e}")
        
        if error < tol:
            break
            
        x = x_new
        lambda_prev = lambda_approx
    
    return lambda_approx, x, k + 1

## Ввод

In [479]:
print("Загружаем матрицы...")
matrices = {}
matrix_files = {
    "inverse1": "hw3/inverse1.txt", 
    "inverse2": "hw3/inverse2.txt", 
    "inverse3": "hw3/inverse3.txt"
}

for name, filename in matrix_files.items():
    matrices[name] = load_matrix_from_file(filename)
print("Матрицы успешно загружены")

Загружаем матрицы...
Матрицы успешно загружены


## Результаты

In [480]:
max_iter = 1000
precision = 1e-12
a = 3.7

print("Точность: ", precision)
print("Значение a: ", a)

for name, matrix in matrices.items():
    print(f"\n{'='*60}")
    print(f"АНАЛИЗ МАТРИЦЫ {name.upper()}")
    print(f"{'='*60}")
    
    dim = matrix.shape
    print(f"Размерность матрицы: {dim}")
    
    initial_vector = np.arange(1, dim[0] + 1, dtype=float)

    eigenvalues = np.linalg.eig(matrix)[0]
    print("Все собственные значения:", sorted(eigenvalues))
        
    print(f"\nПоиск собственного значения ближайшего к {a}:")
    lambda_found, eigenvector, iterations = inverse_iteration(matrix, a, initial_vector, max_iter, precision)
    
    print(f"\nНайденное значение: {lambda_found:.8f}")
    print(f"Итераций: {iterations}")


Точность:  1e-12
Значение a:  3.7

АНАЛИЗ МАТРИЦЫ INVERSE1
Размерность матрицы: (20, 20)
Все собственные значения: [3.0223383475497427, 3.0888543884277215, 3.1980622641951593, 3.347522451368015, 3.533896256340344, 3.753020396282529, 3.999999999999998, 4.269317951267207, 4.554958132087369, 4.850539812827144, 5.14946018717285, 5.4450418679126305, 5.730682048732788, 6.0, 6.246979603717468, 6.466103743659652, 6.6524775486319845, 6.801937735804837, 6.911145611572287, 6.977661652450272]

Поиск собственного значения ближайшего к 3.7:

Найденное значение: 3.75302040
Итераций: 14

АНАЛИЗ МАТРИЦЫ INVERSE2
Размерность матрицы: (20, 20)
Все собственные значения: [0.2538058170966425, 1.789321352666954, 2.961058880693561, 3.996047997334642, 4.999774319814828, 5.999991841327052, 6.99999979492957, 7.999999996191878, 8.999999999945517, 9.999999999999368, 11.000000000000613, 12.00000000005448, 13.000000003808132, 14.000000205070453, 15.000008158672962, 16.00022568018517, 17.003952002665354, 18.038941119